In [63]:
import requests
import pandas as pd
from datetime import datetime
import ccxt
import sqlite3
import config

In [64]:
ex = 'indodax' 
currency_f = 'idr'
currency_c = 'eth'
pair = currency_c.upper() + '/' +currency_f.upper()
now = datetime.utcnow()


In [65]:
def get_fx(currency_f):
    fx = requests.get('https://apilayer.net/api/live?access_key=a1d6d82a3df7cf7882c9dd2b35146d6e&source=USD&format=1').json()
    return fx['quotes']['USD' + currency_f.upper()]

In [66]:
def get_basic(time):
    now_ts = datetime.timestamp(time)
    fx_name = 'USD/' + currency_f.upper()
    fx_rate = get_fx(currency_f)
    basics = {'utc':[now_ts],
     'exchange':[ex],
    'pair':[pair],
     fx_name:[float(fx_rate)]}
    basics = pd.DataFrame(basics)
    return basics

In [67]:
arg = {
    'apiKey': config.indodax['apiKey'],
    'secret': config.indodax['secret']
}

In [68]:
def get_bal(ex, currency_f):
    ex_instance = eval('ccxt.' + ex)(arg)
    bal = ex_instance.fetch_balance()
    bal_fiat_free = bal[currency_f.upper()]['free']
    bal_eth_free = bal['ETH']['free']
    bal_fiat_used = bal[currency_f.upper()]['used']
    bal_eth_used = bal['ETH']['used']
    bal_fiat_total = bal[currency_f.upper()]['total']
    bal_eth_total = bal['ETH']['total']
    bl = {}
    for i in ('bal_fiat_free',  'bal_fiat_used','bal_fiat_total','bal_eth_free','bal_eth_used','bal_eth_total'):
        bl[i] = locals()[i]
    df = pd.DataFrame({k: [v] for k, v in bl.items()})
    return df
        
        
    

In [69]:


basics = get_basic(now)

bl = get_bal(ex, currency_f)

df = pd.concat([basics,bl],axis =1)

In [70]:
# connect to sql
conn = sqlite3.connect('mysqlite.db')
table_name = "bl_" + ex 

In [71]:
# append
df.to_sql(table_name, conn, if_exists = 'append')

In [72]:
# check 
pd.read_sql_query("SELECT * from {}".format(table_name),conn).tail()

,index,utc,exchange,pair,USD/IDR,bal_fiat_free,bal_fiat_used,bal_fiat_total,bal_eth_free,bal_eth_used,bal_eth_total
0,0,1.587802e+09,indodax,ETH/IDR,15587.9,231506022.0,31568897.0,263074919.0,83.923629,11.4441,95.367729
1,0,1.587802e+09,indodax,ETH/IDR,15587.9,231506064.0,31568855.0,263074919.0,83.923629,11.4441,95.367729


In [56]:
conn.close()